In [5]:
%load_ext autoreload
%autoreload 2
import numpy as np
import torch

from torch_obb import obb_estimate_pca, obb_estimate_dito, obb_points_intersection, obb_vertices
import trimesh

device = 'cuda'

rng = np.random.RandomState(132)

def sample_spherical(npoints, ndim=3):
    vec = np.random.randn(npoints, ndim).astype(np.float32)
    vec /= np.linalg.norm(vec, axis=1)[:, None]
    return vec

points1 = rng.random((18, 3)).astype(np.float32) * np.array([1, 1, 1], dtype=np.float32)
points2 = rng.random((1222, 3)).astype(np.float32) * np.array([1, 5, 2], dtype=np.float32)
points3 = sample_spherical(15, 3) * np.array([2, 5, 5] + np.array([1.0, 0.0, 0.0], dtype=np.float32), dtype=np.float32)
rotation_matrix = trimesh.transformations.rotation_matrix(np.pi/3, [1, 0, 0])
points2 = trimesh.transform_points(points2, rotation_matrix).astype(np.float32)
points = torch.nested.as_nested_tensor([torch.from_numpy(points1), torch.from_numpy(points2), torch.from_numpy(points3)], layout=torch.jagged)
points = points.to(device)
obb_pca = obb_estimate_pca(points)
obb_dito = obb_estimate_dito(points)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
obb_intersection = obb_points_intersection(obb_dito, points, pairwise=True, counts=True)
obb_intersection

tensor([  18, 1222,   15], device='cuda:0')

In [7]:
obb_intersection = obb_points_intersection(obb_dito, points, pairwise=False, counts=True)
obb_intersection

tensor([[  18,    9,   18],
        [  27, 1222,  600],
        [   0,    0,   15]], device='cuda:0')

In [12]:
import pygfx as gfx
from rendercanvas.glfw import GlfwRenderCanvas

_TRIS = [[1, 3, 0],
         [4, 1, 0],
         [0, 3, 2],
         [2, 4, 0],
         [1, 7, 3],
         [5, 1, 4],
         [5, 7, 1],
         [3, 7, 2],
         [6, 4, 2],
         [2, 7, 6],
         [6, 5, 4],
         [7, 5, 6]]
i = 1
canvas = GlfwRenderCanvas(size=(1024, 768), max_fps=60, update_mode="continuous")   
scene = gfx.Scene()
grid = gfx.Grid(
    None,
    gfx.GridMaterial(
        major_step=1,
        minor_step=0.1,
        thickness_space="world",
        major_thickness=0.01,
        minor_thickness=0.001,
        infinite=True,
    ),
    orientation="xz",
)
grid.local.z = 0
scene.add(grid)
background = gfx.Background(None, gfx.BackgroundMaterial('white'))
scene.add(background)

vertices_pca_np = obb_vertices(obb_pca)[i].cpu().numpy()
vertices_dito_np = obb_vertices(obb_dito)[i].cpu().numpy()
points_np = points[i].cpu().numpy()

points_geometry = gfx.Geometry(positions=points_np.astype(np.float32))
pc = gfx.Points(points_geometry, material=gfx.PointsMaterial(size=10, color=(0, 0, 0, 1.0)))
scene.add(pc)

obb_geometry_pca = gfx.Geometry(positions=vertices_pca_np.squeeze().astype(np.float32), indices=np.array(_TRIS, dtype=np.int32))
obb_pca_s = gfx.Mesh(obb_geometry_pca, gfx.MeshBasicMaterial(wireframe=True, color='red', wireframe_thickness=2))
scene.add(obb_pca_s)

obb_geometry_dito = gfx.Geometry(positions=vertices_dito_np.squeeze().astype(np.float32), indices=np.array(_TRIS, dtype=np.int32))
obb_dito_s = gfx.Mesh(obb_geometry_dito, gfx.MeshBasicMaterial(wireframe=True, color='green', wireframe_thickness=2))
scene.add(obb_dito_s)

disp = gfx.Display(canvas=canvas)
disp.show(scene)

canvas.is_closed() is deprecated, use canvas.get_closed() instead.
Your scene does not contain any lights. Some objects may not be visible.
Unrecognized present mode 1000361000
Unrecognized present mode 1000361000


Unrecognized present mode 1000361000
